In [152]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np

from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
# from sklearn.datasets import make_classification

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

## Impoetação dos dados

In [153]:
df = pd.read_csv('data.csv')
df = df.drop(['id', 'Unnamed: 32'], axis=1)
df.diagnosis = df.diagnosis.transform(lambda x: 0 if x == 'M' else 1)

## Normalização dos dados

In [154]:
col = df.columns
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled, columns=col)
df.head(5)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,0.0,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,0.686364,...,0.620776,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864
1,0.0,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,...,0.606901,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878
2,0.0,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,...,0.556386,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433
3,0.0,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,...,0.248310,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711
4,0.0,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,...,0.519744,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,0.0,0.690000,0.428813,0.678668,0.566490,0.526948,0.296055,0.571462,0.690358,0.336364,...,0.623266,0.383262,0.576174,0.452664,0.461137,0.178527,0.328035,0.761512,0.097575,0.105667
565,0.0,0.622320,0.626987,0.604036,0.474019,0.407782,0.257714,0.337395,0.486630,0.349495,...,0.560655,0.699094,0.520892,0.379915,0.300007,0.159997,0.256789,0.559450,0.198502,0.074315
566,0.0,0.455251,0.621238,0.445788,0.303118,0.288165,0.254340,0.216753,0.263519,0.267677,...,0.393099,0.589019,0.379949,0.230731,0.282177,0.273705,0.271805,0.487285,0.128721,0.151909
567,0.0,0.644564,0.663510,0.665538,0.475716,0.588336,0.790197,0.823336,0.755467,0.675253,...,0.633582,0.730277,0.668310,0.402035,0.619626,0.815758,0.749760,0.910653,0.497142,0.452315


## Separação dos dados em treino e teste

In [155]:
# Separated the class variable 'y' and the others variables
y = df.diagnosis.values
X = df.drop(['diagnosis'], axis=1).values

# Separate test and training data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Create SVM

In [156]:
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

pred = clf.predict(X_test)
c = 0
for i in range(len(pred)):
    if(y_test[i] == pred[i]):
        c+=1
        
print((c/len(pred)*100))

98.24561403508771


## Random Forest

In [157]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)

pred = clf.predict(X_test)
c = 0
for i in range(len(pred)):
    if(y_test[i] == pred[i]):
        c+=1

print((c/len(pred)*100))

94.73684210526315


## Decision Tree

In [161]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)

pred = clf.predict(X_test)
c = 0
for i in range(len(pred)):
    if(y_test[i] == pred[i]):
        c+=1

print((c/len(pred)*100))

95.32163742690058


## KERAS

In [158]:
#criar o modelo de forma sequencial
model = Sequential()
# train_y = to_categorical(df.High)

#apanhar o numero de colunas no training data
n_cols = len(df.columns) -1

#adicionar camadas ao modelo
model.add(Dense(30, activation='relu', input_shape=(n_cols,)))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='softmax'))

In [159]:
#compilar o modelo usando accuracy para medir a performance
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [160]:
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)
#treinar o modelo

m=model.fit(X_train, y_train, validation_split=0.2, epochs=30)# callbacks=[early_stopping_monitor])

Train on 318 samples, validate on 80 samples
Epoch 1/30
318/318 [==============================] - 1s 2ms/step - loss: 0.3648 - accuracy: 0.6352 - val_loss: 0.4000 - val_accuracy: 0.6000
Epoch 2/30
318/318 [==============================] - 0s 958us/step - loss: 0.3648 - accuracy: 0.6352 - val_loss: 0.4000 - val_accuracy: 0.6000
Epoch 3/30
318/318 [==============================] - 0s 854us/step - loss: 0.3648 - accuracy: 0.6352 - val_loss: 0.4000 - val_accuracy: 0.6000
Epoch 4/30
318/318 [==============================] - 0s 940us/step - loss: 0.3648 - accuracy: 0.6352 - val_loss: 0.4000 - val_accuracy: 0.6000
Epoch 5/30
318/318 [==============================] - 0s 854us/step - loss: 0.3648 - accuracy: 0.6352 - val_loss: 0.4000 - val_accuracy: 0.6000
Epoch 6/30
318/318 [==============================] - 0s 1ms/step - loss: 0.3648 - accuracy: 0.6352 - val_loss: 0.4000 - val_accuracy: 0.6000
Epoch 7/30
318/318 [==============================] - 0s 951us/step - loss: 0.3648 - accuracy: 